<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Preprocesamiento de datos y creación de datasets para forecast</h4>
</center>

In [1]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

In [2]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

# Versión modelo
data_version = '2'
grouped_cols = ["product_id", "cluster"]
grouped_cols_name = "product_cluster"

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [3]:
# Conexiones AWS

account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')

In [4]:
# Buckets de datos

target_s3_path=f's3://datasets-forecast/modelo-{data_version}-forecast-{grouped_cols_name}/target/df_ventas_{grouped_cols_name}.csv'
items_s3_path=f's3://datasets-forecast/modelo-{data_version}-forecast-{grouped_cols_name}/product/df_products.csv'
related_s3_path=f's3://datasets-forecast/modelo-{data_version}-forecast-{grouped_cols_name}/related/df_related_{grouped_cols_name}.csv'

In [5]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

### Procesamiento de datos de ventas

- Skus a predecir

In [6]:
skus_a_predecir = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/productos_a_predecir.txt", delimiter='\t')
skus_a_predecir = skus_a_predecir.product_id.to_list()

- Clusters cliente

In [7]:
clusters = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/cluster_cliente.csv", delimiter=';')
clusters.head(1)

,customer_id,cluster
0,10001,J


- Datos de ventas

In [8]:
df_ventas = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/sell-in.txt", delimiter='\t')
df_ventas['periodo'] = df_ventas['periodo'].astype(str)
df_ventas['periodo'] = pd.to_datetime(df_ventas['periodo'], format='%Y%m')
df_ventas = df_ventas[df_ventas.product_id.isin(skus_a_predecir)]
df_ventas.head(1)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,2017-01-01,10234,20524,0,2,0.053,0.053


In [9]:
print(f"Cantidad productos finales a predecir: {len(df_ventas.product_id.drop_duplicates())}")

Cantidad productos finales a predecir: 780


Dataset de ventas para predecir por periodo, producto y cluster de cliente:

In [10]:
df_ventas_product_client = pd.merge(df_ventas, clusters, how = "left", left_on = "customer_id", right_on = "customer_id")
# Como para armar los clusters tomamos 2019, aquellos clientes que no compran en 2019 no tiene cluster asociado
df_ventas_product_client[df_ventas_product_client.cluster.isna()].periodo.max()

Timestamp('2018-12-01 00:00:00')

In [11]:
# Calculamos cuantos son los clientes inactivos
clientes_act_inact = df_ventas_product_client.drop_duplicates(subset = "customer_id")[["customer_id", "cluster"]]
clientes_act_inact["activo"] = np.where(clientes_act_inact.cluster.isna(), 0, 1)
clientes_act_inact = clientes_act_inact.drop(columns = "cluster")
print("Clientes activos 2019: " + str(len(clientes_act_inact[clientes_act_inact.activo == 1])))
print("Clientes inactivos 2019: " + str(len(clientes_act_inact[clientes_act_inact.activo == 0])))

Clientes activos 2019: 534
Clientes inactivos 2019: 63


In [12]:
# Para no perder el volumen de los clientes sin cluster asociado repartimos el volumen de estos entre los clientes activos en 2019
# Haremos la repartida por producto y por periodo para no gnerar distorsiones

df_reparto = pd.merge(df_ventas_product_client, clientes_act_inact, how = "left", left_on = "customer_id", right_on = "customer_id")
df_reparto = df_reparto.groupby(["periodo", "customer_id", "activo", "product_id"])["tn"].sum().reset_index()

# Filtra los clientes activos y desaparecidos
clientes_activos = df_reparto[df_reparto['activo'] == 1]
clientes_desaparecidos = df_reparto[df_reparto['activo'] == 0]

# Inicializa un DataFrame para almacenar los resultados finales
resultados_finales = pd.DataFrame()

# Itera sobre cada producto y periodo
for product_id in df_reparto['product_id'].unique():
    for periodo in df_reparto['periodo'].unique():
        
        # Filtra los datos por producto y periodo
        activos_producto_periodo = clientes_activos[(clientes_activos['product_id'] == product_id) & (clientes_activos['periodo'] == periodo)]
        desaparecidos_producto_periodo = clientes_desaparecidos[(clientes_desaparecidos['product_id'] == product_id) & (clientes_desaparecidos['periodo'] == periodo)]
        
        # Busca los periodos en común: aqui se pierde año 2019 en donde los desaparecidos no tienen venta
        if len(activos_producto_periodo) > 0 and len(desaparecidos_producto_periodo) > 0:
            # Calcula el total de ventas de clientes activos para el producto y periodo
            total_tn_activos = activos_producto_periodo['tn'].sum()
            
            # Calcula el total de ventas de clientes desaparecidos para el producto y periodo
            total_tn_desaparecidos = desaparecidos_producto_periodo['tn'].sum()
            
            # Calcula el porcentaje de ventas de cada cliente activo para el producto y periodo
            activos_producto_periodo['porcentaje_tn'] = activos_producto_periodo['tn'] / total_tn_activos
            
            # Distribuye el volumen de ventas de los clientes desaparecidos entre los clientes activos
            activos_producto_periodo['tn_asignadas'] = activos_producto_periodo['porcentaje_tn'] * total_tn_desaparecidos
            
            # Suma las ventas originales y las asignadas
            activos_producto_periodo['tn_totales'] = activos_producto_periodo['tn'] + activos_producto_periodo['tn_asignadas']
            
            # Añade los resultados al DataFrame final
            resultados_finales = pd.concat([resultados_finales, activos_producto_periodo])

# Resultado final
resultados_finales[['periodo', 'customer_id', 'product_id', 'tn', 'tn_asignadas', 'tn_totales']].head(1)
# DF CON LOS CLIENTES ACTIVOS Y SUS TN ASIGNADAS DE LOS INACTIVOS

,periodo,customer_id,product_id,tn,tn_asignadas,tn_totales
0,2017-01-01,10001,20001,99.43861,1.683058,101.121668


In [13]:
df_reparto_final = pd.merge(clientes_activos, resultados_finales[['periodo', 'customer_id', 'product_id', 'tn_asignadas', 'tn_totales']],
                            on=['periodo', 'customer_id', 'product_id'], how='left')
# Rellena NaN en 'tn_asignadas' y 'tn_totales' con 0 y el valor original de 'tn', respectivamente
df_reparto_final['tn_asignadas'] = df_reparto_final['tn_asignadas'].fillna(0)
df_reparto_final['tn_totales'] = df_reparto_final['tn_totales'].fillna(df_reparto_final['tn'])
df_reparto_final = df_reparto_final.drop(columns = {"activo", "tn", "tn_asignadas"})
df_reparto_final = df_reparto_final.rename(columns = {"tn_totales": "tn"})
df_reparto_final.head(1)

,periodo,customer_id,product_id,tn
0,2017-01-01,10001,20001,101.121668


In [14]:
df_ventas_product_cluster = df_reparto_final.copy()
df_ventas_product_cluster = pd.merge(df_ventas_product_cluster, clusters, how = "left", left_on = "customer_id", right_on = "customer_id")
df_ventas_product_cluster = df_ventas_product_cluster.groupby(["periodo", "cluster", "product_id"])["tn"].sum().reset_index()
df_ventas_product_cluster.head(1)

,periodo,cluster,product_id,tn
0,2017-01-01,A,20001,0.170742


In [15]:
df_ventas_product_cluster.isna().sum()

periodo       0
cluster       0
product_id    0
tn            0
dtype: int64

In [16]:
df_ventas_product_cluster.tn.sum().round(1) == df_ventas.tn.sum().round(1) 

True

In [17]:
# Exportamos data de ventas product cluster
df_ventas_product_cluster_export = df_ventas_product_cluster[(df_ventas_product_cluster['periodo']>=date_start) & (df_ventas_product_cluster['periodo']<=date_end)]
df_ventas_product_cluster_export['product_id'] = df_ventas_product_cluster_export['product_id'].astype(str)
df_ventas_product_cluster_export = df_ventas_product_cluster_export[["periodo", "product_id", "cluster", "tn"]]
print(f"Min periodo {df_ventas_product_cluster_export.periodo.min()}")
print(f"Max periodo {df_ventas_product_cluster_export.periodo.max()}")
print(f"Tipos por columna \n{df_ventas_product_cluster_export.dtypes}")
wr.s3.to_csv(df_ventas_product_cluster_export, target_s3_path, index=False, header=False)

Min periodo 2017-01-01 00:00:00
Max periodo 2019-12-01 00:00:00
Tipos por columna 
periodo       datetime64[ns]
product_id            object
cluster               object
tn                   float64
dtype: object


{'paths': ['s3://datasets-forecast/modelo-2-forecast-product_cluster/target/df_ventas_product_cluster.csv'],
 'partitions_values': {}}

### Procesamiento de datos de productos

In [18]:
producto = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/tb_productos.txt", delimiter='\t')
producto.head(5)

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288


In [19]:
print("----ANALISIS DUPLICADOS-----")
print(f"Cantidad productos en maestro productos: {len(producto)}")
print(f"Cantidad productos en maestro productos sin repetidos: {len(producto.product_id.drop_duplicates())}")
# Observamos que product_id duplicados tienen los mismos valores en las otras columnas, son duplicados y procedemos a eliminarlos 
producto = producto.drop_duplicates(subset = "product_id")
print(f"Cantidad productos en maestro productos: {len(producto)}")

----ANALISIS DUPLICADOS-----
Cantidad productos en maestro productos: 1262
Cantidad productos en maestro productos sin repetidos: 1251
Cantidad productos en maestro productos: 1251


In [20]:
producto = producto[producto.product_id.isin(skus_a_predecir)]
print(f"Cantidad productos finales a predecir: {len(producto)}")

Cantidad productos finales a predecir: 780


In [21]:
producto = producto[["product_id", "cat1", "cat2", "cat3", "brand", "sku_size"]]
producto['product_id'] = producto['product_id'].astype(str)
print(f"Tipos por columna \n{producto.dtypes}")
wr.s3.to_csv(producto, items_s3_path, index=False, header=False)

Tipos por columna 
product_id    object
cat1          object
cat2          object
cat3          object
brand         object
sku_size       int64
dtype: object


{'paths': ['s3://datasets-forecast/modelo-2-forecast-product_cluster/product/df_products.csv'],
 'partitions_values': {}}

### Procesamiento de datos relacionados

In [22]:
def create_future_periods_cluster(df):
    combination = df.groupby([i for i in ['product_id', 'cluster']])["periodo"].apply(lambda s: pd.date_range(s.min(), date_future, freq='MS')).explode().reset_index()
    result = pd.merge(combination, df, how = "left", on = ["periodo", "cluster", "product_id"])
    result = result[["periodo", "product_id", "cluster", "tn"]]
    return result

In [23]:
df_related_product_cluster_v1 = create_future_periods_cluster(df_ventas_product_cluster)

In [24]:
def time_features(df):
    df_time_features = df.copy()
    # Asegúrate de que la columna 'periodo' esté en formato datetime
    df_time_features['periodo'] = pd.to_datetime(df_time_features['periodo'], format='%Y-%m-%d')
    # Crear las columnas 'mes', 'cuatrimestre' y 'dias del mes"
    df_time_features['month'] = df_time_features['periodo'].dt.month
    df_time_features['quarter'] = df_time_features['periodo'].dt.quarter
    df_time_features['days_in_month']= df_time_features['periodo'].dt.daysinmonth
    return df_time_features

In [25]:
df_related_product_cluster_v2 = time_features(df_related_product_cluster_v1)

In [26]:
# Cantidad de domingos al mes
sundays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start), end=date_future + pd.offsets.MonthEnd(0)))
# sundays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start) - pd.DateOffset(years=1), end=date_future + pd.offsets.MonthEnd(0)))
sundays.columns = ["fecha"]
sundays["day_week"] = pd.to_datetime(sundays['fecha'], format='%Y-%m-%d').dt.dayofweek
sundays = sundays[sundays.day_week == 6].rename(columns = {"day_week":"sundays"})
sundays = sundays.replace({6:1})
sundays['periodo']=sundays['fecha'].apply(lambda x: x.replace(day=1))
sundays = sundays.groupby(["periodo"])["sundays"].sum().reset_index()
sundays['periodo']=pd.to_datetime(sundays['periodo'], format='%Y-%m-%d')
sundays["sundays"] = sundays["sundays"].astype(int)
# Cantidad de sabados al mes
saturdays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start), end=date_future + pd.offsets.MonthEnd(0)))
# saturdays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start) - pd.DateOffset(years=1), end=date_future + pd.offsets.MonthEnd(0)))
saturdays.columns = ["fecha"]
saturdays["day_week"] = pd.to_datetime(saturdays['fecha'], format='%Y-%m-%d').dt.dayofweek
saturdays = saturdays[saturdays.day_week == 5].rename(columns = {"day_week":"saturdays"})
saturdays = saturdays.replace({5:1})
saturdays['periodo']=saturdays['fecha'].apply(lambda x: x.replace(day=1))
saturdays = saturdays.groupby(["periodo"])["saturdays"].sum().reset_index()
saturdays["saturdays"] = saturdays["saturdays"].astype(int)
saturdays['periodo']=pd.to_datetime(saturdays['periodo'], format='%Y-%m-%d')
# DF conjunto
sundays_saturdays = pd.merge(sundays, saturdays, how = "inner", on = "periodo")
sundays_saturdays.head(5)

,periodo,sundays,saturdays
0,2017-01-01,5,4
1,2017-02-01,4,4
2,2017-03-01,4,4
3,2017-04-01,5,5
4,2017-05-01,4,4


In [27]:
df_related_product_cluster_v3 = pd.merge(df_related_product_cluster_v2, sundays_saturdays, how = "left", on = "periodo")

In [28]:
def shift_series(df, columns_shift):
    df_shift = df.copy()
    # Ordenar el DataFrame por product_id y periodo para aplicar shift correctamente
    df_shift = df_shift.sort_values(by = columns_shift + ["periodo"])
    df_shift['tn_m3'] = df_shift.groupby(columns_shift)['tn'].shift(3)
    df_shift['tn_m6'] = df_shift.groupby(columns_shift)['tn'].shift(6)
    df_shift['tn_m12'] = df_shift.groupby(columns_shift)['tn'].shift(12)
    df_shift = df_shift.drop(columns = "tn")
    return df_shift

In [29]:
df_related_product_cluster_v4 = shift_series(df_related_product_cluster_v3, columns_shift = ["cluster", "product_id"])

In [30]:
# Exportamos data relacionada, apertura por product y cluster
# Nos aseguramos que cada serie temporal (por producto y cluster) comience en el mismo periodo que comienza en la serie de tiempo ppal de ventas
df_min_period = df_ventas_product_cluster_export.groupby(["product_id", "cluster"])["periodo"].min().reset_index().rename(columns = {"periodo":"min_periodo"})
df_related_product_cluster_v4['product_id'] = df_related_product_cluster_v4['product_id'].astype(str)
df_related_product_cluster_export = pd.merge(df_related_product_cluster_v4, df_min_period, on=['product_id', 'cluster'])
df_related_product_cluster_export = df_related_product_cluster_export[df_related_product_cluster_export['periodo'] >= df_related_product_cluster_export['min_periodo']]
df_related_product_cluster_export = df_related_product_cluster_export.drop(columns=['min_periodo'])
df_related_product_cluster_export = df_related_product_cluster_export[(df_related_product_cluster_export['periodo']<=date_future)]
df_related_product_cluster_export['product_id'] = df_related_product_cluster_export['product_id'].astype(str)
print("-------------------------")
print(f"Min periodo {df_related_product_cluster_export.periodo.min()}")
print(f"Max periodo {df_related_product_cluster_export.periodo.max()}")
print(f"Cantidad de productos {len(df_related_product_cluster_export.product_id.unique())}")
print("-------------------------")
print(f"Tipos por columna \n{df_related_product_cluster_export.dtypes}")
print("-------------------------")
print(f"Cantidad de NAN values \n{df_related_product_cluster_export.isna().sum()}")
print("-------------------------")
wr.s3.to_csv(df_related_product_cluster_export, related_s3_path, index=False, header=False)

-------------------------
Min periodo 2017-01-01 00:00:00
Max periodo 2020-02-01 00:00:00
Cantidad de productos 780
-------------------------
Tipos por columna 
periodo          datetime64[ns]
product_id               object
cluster                  object
month                     int32
quarter                   int32
days_in_month             int32
sundays                   int32
saturdays                 int32
tn_m3                   float64
tn_m6                   float64
tn_m12                  float64
dtype: object
-------------------------
Cantidad de NAN values 
periodo              0
product_id           0
cluster              0
month                0
quarter              0
days_in_month        0
sundays              0
saturdays            0
tn_m3            37620
tn_m6            58727
tn_m12           98166
dtype: int64
-------------------------


{'paths': ['s3://datasets-forecast/modelo-2-forecast-product_cluster/related/df_related_product_cluster.csv'],
 'partitions_values': {}}

Chequear que cada combinacion en el df relacionado comience en misma fecha que en df principal para evitar errores en entrenamiento

In [31]:
fecha_min_ventas = df_ventas_product_cluster_export.groupby(["product_id", "cluster"])["periodo"].min().reset_index().rename(columns = {"periodo":"min_period_ventas"})
fecha_min_related = df_related_product_cluster_export.groupby(["product_id", "cluster"])["periodo"].min().reset_index().rename(columns = {"periodo":"min_period_related"})
merge_fecha_min = pd.merge(fecha_min_ventas, fecha_min_related, how = "outer", on = ["product_id", "cluster"])
merge_fecha_min[merge_fecha_min.min_period_ventas != merge_fecha_min.min_period_related]

,product_id,cluster,min_period_ventas,min_period_related
